In [1]:
import geopandas
from openai import OpenAI

import os
import wget
import zipfile

from dotenv import load_dotenv

from hackathonopenai.national_monuments import NationalMonumentsAssistant

%load_ext autoreload
%autoreload 2

In [2]:

def download_national_parks_data(
        url_path: str = "https://opendata.arcgis.com/datasets/beb04b0e5bed44da9e5407ea50477a5c_0.zip",
        output_directory_base: str = "../data/parques_nacionales",
        raw_data: str = '../data/raw',
        name: str = "data_parques"
) -> str:
    """
    Download the national parks data from the url_path and save it in the output_directory_base
    """
    if not os.path.exists(output_directory_base):
        os.makedirs(output_directory_base, exist_ok=True)

    if not os.path.exists(raw_data):
        os.makedirs(raw_data, exist_ok=True)

    # Download the data
    output_directory = wget.download(url_path, out=raw_data)
    # Unzip the data
    with zipfile.ZipFile(output_directory, 'r') as zip_ref:
        zip_ref.extractall(output_directory_base)

    for file in os.listdir(output_directory_base):
        os.rename(
            os.path.join(output_directory_base, file), 
            os.path.join(output_directory_base, name + os.path.splitext(file)[1])
        )
    return output_directory_base

In [3]:
load_dotenv("../.env")

name = "data_parques"
output_directory_base = download_national_parks_data(name=name)

# Load files
df = geopandas.read_file(
    os.path.join(output_directory_base, name + ".shp")
)
df_demo = geopandas.read_file(
    "../data/demo_data/doc.kml"
)

In [4]:
from shapely.ops import unary_union

single_polygon = unary_union(df_demo.geometry)
single_polygon_df = geopandas.GeoDataFrame(geometry=[single_polygon], crs=df_demo.crs)

In [5]:
single_polygon_df

,geometry
0,"POLYGON Z ((-71.4334 -33.29117 0, -71.4336 -33..."


In [6]:
df_kms = df.to_crs("EPSG:32633")
demo_geometry_kms = df_demo.to_crs("EPSG:32633")

In [7]:
park_assistant = NationalMonumentsAssistant(df_kms, OpenAI())

In [30]:
response = park_assistant.evaluate_project(demo_geometry_kms, threshold_in_kilometers=10)

Numero de parques cercanos:  0


In [31]:
close_parks = response["close_parks"]

In [42]:
import folium
location = demo_geometry_kms

location.to_crs("EPSG:4326", inplace=True)
close_parks.to_crs("EPSG:4326", inplace=True)

centroid = location.centroid.iloc[0]
center = [centroid.y, centroid.x]

mapa = folium.Map(location=center, zoom_start=12)
# Agregar los polígonos al mapa
folium.GeoJson(
    close_parks[['NOMBRE','geometry']],
    style_function=lambda x: {
        "fillColor": "red",
        "color": "black",
        "weight": 2.5,
        "fillOpacity": 0.5,
        "tooltip": x["properties"]["NOMBRE"],
    },
).add_to(mapa)

folium.GeoJson(
    location,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "weight": 2.5,
        "fillOpacity": 0.5,
    },
).add_to(mapa)

/var/folders/yz/zfvnc9qs3557ss5cr7wv0c8xvzxvfn/T/ipykernel_58843/776072422.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = location.centroid.iloc[0]


In [43]:
mapa